In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import xgboost as xgb
import psutil
from statsmodels.tsa.stattools import acf, pacf, adfuller, q_stat
from statsmodels.graphics.tsaplots import plot_acf
import matplotlib.pyplot as plt
from pmdarima import auto_arima
from prophet import Prophet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from joblib import Parallel, delayed
import warnings
import os
warnings.filterwarnings("ignore")

In [2]:
TRAIN_CSV = r"D:\DEPI\CIS project\Train.csv"
TEST_CSV = r"D:\DEPI\CIS project\Test.csv"
SUB_CSV = r"D:\DEPI\CIS project\Submission.csv"
TRAIN_END = '2017-07-15'
VAL_END = '2017-08-15'

In [3]:
print("Loading data...")
train = pd.read_csv(TRAIN_CSV)
test = pd.read_csv(TEST_CSV)
sub = pd.read_csv(SUB_CSV)
for df in (train, test):
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d' if df is train else '%d-%m-%Y', errors='coerce')
    df[['store_nbr', 'onpromotion']] = df[['store_nbr', 'onpromotion']].astype('int32')
    df['sales'] = df['sales'].astype('float32') if 'sales' in df else np.nan
    df.dropna(subset=['date'], inplace=True)
if train.empty:
    raise ValueError("Train data is empty after preprocessing.")

Loading data...


In [ ]:
train['is_train'] = 1
test['is_train'] = 0
combined = pd.concat([train, test]).sort_values(['store_nbr', 'family', 'date'])
agg_dict = {col: 'first' for col in combined.columns if col not in ['store_nbr', 'family', 'date', 'sales', 'onpromotion', 'is_train']}
agg_dict.update({'sales': 'mean', 'onpromotion': 'sum', 'is_train': 'first'})
combined = combined.groupby(['store_nbr', 'family', 'date']).agg(agg_dict).reset_index()
combined = combined.astype({
    'store_nbr': 'int32',
    'family': 'category',
    'date': 'datetime64[ns]',
    'sales': 'float32',
    'onpromotion': 'int32',
    'is_train': 'int8'
})
combined = combined.set_index(['store_nbr', 'family', 'date'])

Preparing data...


In [ ]:
grouped = combined.groupby(['store_nbr', 'family'])
processed_groups = []
for (store_nbr, family), group in grouped:
    # Reset index, removing 'store_nbr' and 'family' from index but not adding as columns yet
    group = group.reset_index(['store_nbr', 'family'], drop=True)
    group.index = pd.to_datetime(group.index)  # Ensure 'date' index is datetime
    group = group[~group.index.duplicated()]  # Remove duplicate dates
    
    # Handle missing values
    group['sales'] = group['sales'].ffill().fillna(0).astype('float32')
    group['onpromotion'] = group['onpromotion'].fillna(0).astype('int32')
    for col in group:
        if col not in ['sales', 'onpromotion']:
            group[col] = group[col].ffill().fillna(group[col].iloc[0] if not group[col].isna().all() else 0)
    
    # Add 'store_nbr' and 'family' as columns using the group key
    group['store_nbr'] = store_nbr
    group['family'] = family
    
    # Move 'date' from index to column
    group = group.reset_index()
    
    processed_groups.append(group)

# Concatenate all processed groups
combined = pd.concat(processed_groups)

# Set the index to ['store_nbr', 'family', 'date']
combined = combined.set_index(['store_nbr', 'family', 'date'])

Handling missing values...


In [ ]:
combined = combined.reset_index()
combined['day'] = combined['date'].dt.day.astype('int8')
combined['dow'] = combined['date'].dt.dayofweek.astype('int8')
combined['is_weekend'] = combined['dow'].isin([5, 6]).astype('int8')
combined['woy'] = combined['date'].dt.isocalendar().week.astype('int8')
combined['month'] = combined['date'].dt.month.astype('int8')
combined['quarter'] = combined['date'].dt.quarter.astype('int8')
combined['year'] = combined['date'].dt.year.astype('int16')
combined['sin_month'] = np.sin(2 * np.pi * combined['month'] / 12).astype('float32')
combined['cos_month'] = np.cos(2 * np.pi * combined['month'] / 12).astype('float32')
combined['promo_weekend'] = (combined['onpromotion'] * combined['is_weekend']).astype('int8')
combined['description'] = combined['description'].astype(str)
combined['is_holiday'] = combined['description'].str.contains("Holiday|Navidad|Carnaval", case=False, na=False).astype('int8')

lags = [1, 7, 14, 21, 28]
windows = [7, 14, 21, 28]
for lag in lags:
    combined[f'lag_{lag}'] = combined.groupby(['store_nbr', 'family'])['sales'].shift(lag).astype('float32')
for w in windows:
    roll = combined.groupby(['store_nbr', 'family'])['sales'].shift(1).rolling(w, min_periods=1)
    combined[f'roll_mean_{w}'] = roll.mean().astype('float32')
    combined[f'roll_std_{w}'] = roll.std().astype('float32')
combined['diff_1'] = combined.groupby(['store_nbr', 'family'])['sales'].diff(1).astype('float32')

combined['store_nbr_encoded'] = LabelEncoder().fit_transform(combined['store_nbr']).astype('int8')
combined['family_encoded'] = LabelEncoder().fit_transform(combined['family']).astype('int8')

feature_cols = ['onpromotion', 'promo_weekend', 'is_holiday', 'day', 'dow', 'is_weekend', 'woy', 'month',
                'quarter', 'year', 'sin_month', 'cos_month', 'store_nbr_encoded', 'family_encoded'] + \
               [f'lag_{lag}' for lag in lags] + [f'roll_mean_{w}' for w in windows] + \
               [f'roll_std_{w}' for w in windows] + ['diff_1']
combined[feature_cols] = StandardScaler().fit_transform(combined[feature_cols].fillna(0)).astype('float32')
combined = combined.set_index(['store_nbr', 'family', 'date'])

Adding features...


In [9]:
train = combined[combined['is_train'] == 1].drop('is_train', axis=1)
test = combined[combined['is_train'] == 0].drop(['is_train', 'sales'], axis=1)
train_set = train[train.index.get_level_values('date') <= TRAIN_END]
val_set = train[(train.index.get_level_values('date') > TRAIN_END) & (train.index.get_level_values('date') <= VAL_END)].dropna(subset=['sales'])
if val_set.empty:
    raise ValueError("Validation set is empty.")

In [10]:
print("Training XGBoost...")
X_train = train_set[feature_cols]
y_train = train_set['sales']
X_val = val_set[feature_cols]
y_val = val_set['sales']

model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    random_state=42,
    eval_metric='rmse'
)

early_stop = xgb.callback.EarlyStopping(
    rounds=10,
    metric_name='rmse',
    data_name='validation_0',
    save_best=True
)

model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=True
)

y_pred_val = model.predict(X_val)

actual = np.clip(y_val, 0, None)
predicted = np.clip(y_pred_val, 0, None)
rmsle = np.sqrt(mean_squared_error(np.log1p(actual), np.log1p(predicted)))
rmse = np.sqrt(mean_squared_error(actual, predicted))
mae = mean_absolute_error(actual, predicted)
mape = mean_absolute_percentage_error(actual + 1e-10, predicted + 1e-10)
print(f"XGBoost Metrics: RMSLE={rmsle:.4f}, RMSE={rmse:.4f}, MAE={mae:.4f}, MAPE={mape:.4f}")

Training XGBoost...
[0]	validation_0-rmse:989.44318
[1]	validation_0-rmse:892.77377
[2]	validation_0-rmse:804.66383
[3]	validation_0-rmse:723.72571
[4]	validation_0-rmse:651.81157
[5]	validation_0-rmse:588.50992
[6]	validation_0-rmse:527.87906
[7]	validation_0-rmse:474.50653
[8]	validation_0-rmse:426.43887
[9]	validation_0-rmse:384.56282
[10]	validation_0-rmse:345.99290
[11]	validation_0-rmse:312.03650
[12]	validation_0-rmse:280.88012
[13]	validation_0-rmse:254.99973
[14]	validation_0-rmse:230.32120
[15]	validation_0-rmse:209.97231
[16]	validation_0-rmse:192.05279
[17]	validation_0-rmse:175.64734
[18]	validation_0-rmse:161.06883
[19]	validation_0-rmse:148.85556
[20]	validation_0-rmse:138.44808
[21]	validation_0-rmse:130.41651
[22]	validation_0-rmse:123.00202
[23]	validation_0-rmse:117.08262
[24]	validation_0-rmse:112.37035
[25]	validation_0-rmse:109.01658
[26]	validation_0-rmse:105.53972
[27]	validation_0-rmse:103.91466
[28]	validation_0-rmse:101.90643
[29]	validation_0-rmse:100.23211


In [11]:
xgb_test_preds = model.predict(test[feature_cols])
test['sales'] = xgb_test_preds
test_copy = test.reset_index()
submission = test_copy[['id', 'sales']].merge(sub[['id']], on='id', how='right').fillna({'sales': 0}).clip(lower=0)
submission.to_csv(r"D:\DEPI\CIS project\submission_xgboost.csv", index=False)
print("XGBoost submission created.")

XGBoost submission created.


In [13]:
for name, df in [('train', train_set), ('val', val_set), ('test', test)]:
    df.select_dtypes(include=[np.number]).to_parquet(f"D:\DEPI\CIS project\{name}_processed.parquet")

In [14]:
train_groups = train_set.groupby(['store_nbr', 'family'])
val_groups = val_set.groupby(['store_nbr', 'family'])
val_dates = pd.date_range('2017-07-16', '2017-08-15')
test_dates = pd.date_range('2017-08-16', '2017-08-31')
val_steps = len(val_dates)
test_steps = len(test_dates)

In [15]:
print("Training ARIMA...")
arima_models = {}
for key, group in train_groups:
    try:
        model = auto_arima(group['sales'], seasonal=False, max_p=5, max_q=5, trace=False, error_action='ignore')
        arima_models[key] = model
    except:
        arima_models[key] = None
arima_val_preds = {}
for k, m in arima_models.items():
    if m:
        arima_val_preds[k] = m.predict(val_steps)
    else:
        arima_val_preds[k] = np.zeros(val_steps)
arima_test_preds = {}
for k, m in arima_models.items():
    if m:
        arima_test_preds[k] = m.predict(test_steps)
    else:
        arima_test_preds[k] = np.zeros(test_steps)

Training ARIMA...


In [ ]:
actuals = []
preds = []
for (s, f), g in val_groups:
    actuals.extend(g['sales'].values)
    preds.extend(arima_val_preds[(s, f)])
actual = np.clip(actuals, 0, None)
predicted = np.clip(preds, 0, None)
rmsle = np.sqrt(mean_squared_error(np.log1p(actual), np.log1p(predicted)))
rmse = np.sqrt(mean_squared_error(actual, predicted))
mae = mean_absolute_error(actual, predicted)
mape = mean_absolute_percentage_error(actual + 1e-10, predicted + 1e-10)
print(f"ARIMA Metrics: RMSLE={rmsle:.4f}, RMSE={rmse:.4f}, MAE={mae:.4f}, MAPE={mape:.4f}")

test_copy = test.reset_index()
for (store, family), pred_values in arima_test_preds.items():
    mask = (test_copy['store_nbr'] == store) & (test_copy['family'] == family)
    test_copy.loc[mask, 'sales'] = pred_values[:mask.sum()]
submission = test_copy[['id', 'sales']].merge(sub[['id']], on='id', how='right').fillna({'sales': 0}).clip(lower=0)
submission.to_csv(r"D:\DEPI\CIS project\submission_arima.csv", index=False)
print("ARIMA submission created.")

In [ ]:
print("Training SARIMA...")
sarima_models = {}
for key, group in train_groups:
    try:
        model = auto_arima(group['sales'], seasonal=True, m=7, max_p=5, max_q=5, trace=False, error_action='ignore')
        sarima_models[key] = model
    except:
        sarima_models[key] = None
sarima_val_preds = {}
for k, m in sarima_models.items():
    if m:
        sarima_val_preds[k] = m.predict(val_steps)
    else:
        sarima_val_preds[k] = np.zeros(val_steps)
sarima_test_preds = {}
for k, m in sarima_models.items():
    if m:
        sarima_test_preds[k] = m.predict(test_steps)
    else:
        sarima_test_preds[k] = np.zeros(test_steps)

In [ ]:
actuals = []
preds = []
for (s, f), g in val_groups:
    actuals.extend(g['sales'].values)
    preds.extend(sarima_val_preds[(s, f)])
actual = np.clip(actuals, 0, None)
predicted = np.clip(preds, 0, None)
rmsle = np.sqrt(mean_squared_error(np.log1p(actual), np.log1p(predicted)))
rmse = np.sqrt(mean_squared_error(actual, predicted))
mae = mean_absolute_error(actual, predicted)
mape = mean_absolute_percentage_error(actual + 1e-10, predicted + 1e-10)
print(f"SARIMA Metrics: RMSLE={rmsle:.4f}, RMSE={rmse:.4f}, MAE={mae:.4f}, MAPE={mape:.4f}")

test_copy = test.reset_index()
for (store, family), pred_values in sarima_test_preds.items():
    mask = (test_copy['store_nbr'] == store) & (test_copy['family'] == family)
    test_copy.loc[mask, 'sales'] = pred_values[:mask.sum()]
submission = test_copy[['id', 'sales']].merge(sub[['id']], on='id', how='right').fillna({'sales': 0}).clip(lower=0)
submission.to_csv(r"D:\DEPI\CIS project\submission_sarima.csv", index=False)
print("SARIMA submission created.")

In [ ]:
print("Training Prophet...")
prophet_models = {}
for key, group in train_groups:
    df = group.reset_index()[['date', 'sales']].rename(columns={'date': 'ds', 'sales': 'y'})
    m = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=True)
    m.fit(df)
    prophet_models[key] = m
prophet_val_preds = {}
for k, m in prophet_models.items():
    prophet_val_preds[k] = m.predict(pd.DataFrame({'ds': val_dates}))['yhat'].values
prophet_test_preds = {}
for k, m in prophet_models.items():
    prophet_test_preds[k] = m.predict(pd.DataFrame({'ds': test_dates}))['yhat'].values

In [ ]:
actuals = []
preds = []
for (s, f), g in val_groups:
    actuals.extend(g['sales'].values)
    preds.extend(prophet_val_preds[(s, f)])
actual = np.clip(actuals, 0, None)
predicted = np.clip(preds, 0, None)
rmsle = np.sqrt(mean_squared_error(np.log1p(actual), np.log1p(predicted)))
rmse = np.sqrt(mean_squared_error(actual, predicted))
mae = mean_absolute_error(actual, predicted)
mape = mean_absolute_percentage_error(actual + 1e-10, predicted + 1e-10)
print(f"Prophet Metrics: RMSLE={rmsle:.4f}, RMSE={rmse:.4f}, MAE={mae:.4f}, MAPE={mape:.4f}")

test_copy = test.reset_index()
for (store, family), pred_values in prophet_test_preds.items():
    mask = (test_copy['store_nbr'] == store) & (test_copy['family'] == family)
    test_copy.loc[mask, 'sales'] = pred_values[:mask.sum()]
submission = test_copy[['id', 'sales']].merge(sub[['id']], on='id', how='right').fillna({'sales': 0}).clip(lower=0)
submission.to_csv(r"D:\DEPI\CIS project\submission_prophet.csv", index=False)
print("Prophet submission created.")

In [ ]:
print("Training LSTM...")
seq_length = 7

X_train = []
y_train = []
for _, g in train_set.groupby(['store_nbr', 'family']):
    g = g.sort_index()
    for i in range(len(g) - seq_length):
        X_train.append(g.iloc[i:i+seq_length][feature_cols].values)
        y_train.append(g.iloc[i+seq_length]['sales'])
X_train = np.array(X_train)
y_train = np.array(y_train)

X_val = []
y_val = []
for _, g in val_set.groupby(['store_nbr', 'family']):
    g = g.sort_index()
    for i in range(len(g) - seq_length):
        X_val.append(g.iloc[i:i+seq_length][feature_cols].values)
        y_val.append(g.iloc[i+seq_length]['sales'])
X_val = np.array(X_val)
y_val = np.array(y_val)

model = Sequential([LSTM(50, activation='relu', input_shape=(seq_length, len(feature_cols))), Dense(1)])
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_val, y_val), verbose=1)

lstm_val_preds = model.predict(X_val).flatten()

X_test = []
for key in test.groupby(['store_nbr', 'family']).groups.keys():
    last_seq = train_set.xs(key, level=['store_nbr', 'family']).tail(seq_length)[feature_cols].values
    X_test.append(last_seq)
X_test = np.array(X_test)

test_preds_raw = model.predict(X_test).flatten()
lstm_test_preds = {}
for i, key in enumerate(test.groupby(['store_nbr', 'family']).groups.keys()):
    group_size = len(test.xs(key[0], level='store_nbr').xs(key[1], level='family'))
    lstm_test_preds[key] = np.full(group_size, test_preds_raw[i])

In [ ]:
actual = np.clip(y_val, 0, None)
predicted = np.clip(lstm_val_preds, 0, None)
rmsle = np.sqrt(mean_squared_error(np.log1p(actual), np.log1p(predicted)))
rmse = np.sqrt(mean_squared_error(actual, predicted))
mae = mean_absolute_error(actual, predicted)
mape = mean_absolute_percentage_error(actual + 1e-10, predicted + 1e-10)
print(f"LSTM Metrics: RMSLE={rmsle:.4f}, RMSE={rmse:.4f}, MAE={mae:.4f}, MAPE={mape:.4f}")

test_copy = test.reset_index()
for (store, family), pred_values in lstm_test_preds.items():
    mask = (test_copy['store_nbr'] == store) & (test_copy['family'] == family)
    test_copy.loc[mask, 'sales'] = pred_values[:mask.sum()]
submission = test_copy[['id', 'sales']].merge(sub[['id']], on='id', how='right').fillna({'sales': 0}).clip(lower=0)
submission.to_csv(r"D:\DEPI\CIS project\submission_lstm.csv", index=False)
print("LSTM submission created.")

In [ ]:
store = 1
family = 'AUTOMOTIVE'
ts = train_set.loc[(store, family), 'sales'].sort_index()
print(f"Analyzing Store {store}, Family {family}...")
plt.figure(figsize=(12, 6))
plt.plot(ts, label='Original')
plt.plot(ts.rolling(30).mean(), label='Rolling Mean', color='red')
plt.plot(ts.rolling(30).std(), label='Rolling Std', color='black')
plt.title(f"Stationarity: Store {store}, Family {family}")
plt.legend()
plt.show()

result = adfuller(ts.dropna())
print(f"ADF Statistic: {result[0]:.4f}, p-value: {result[1]:.4f}")

ts_diff = ts.diff().dropna()
plt.figure(figsize=(12, 6))
plot_acf(ts_diff, lags=20)
plt.title(f"ACF of Differenced Series")
plt.show()

lagged = pd.DataFrame({'sales': ts})
for lag in [1, 7, 14, 21, 28]:
    lagged[f'lag_{lag}'] = ts.shift(lag)
print("Lagged Correlations:\n", lagged.corr()['sales'].drop('sales'))

In [ ]:
print("Execution completed.")
mem = psutil.Process(os.getpid()).memory_info().rss / 1024**2
print(f"Memory usage: {mem:.2f} MB")